In [2]:
# Distance Vector routing (Bellman–Ford)
# - Directed graph
# - Converges by synchronous iterations
# - Tie-break on next-hop by: lower cost, then alphabetically by hop name

from collections import defaultdict
import math

INF = math.inf

class DistanceVector:
    def __init__(self, nodes):
        self.nodes = sorted(nodes)
        self.adj = {u: {} for u in self.nodes}
        # Distance vectors and next-hops per source
        self.dv = {u: {v: (0 if u == v else INF) for v in self.nodes} for u in self.nodes}
        self.nh = {u: {v: (None if u == v else None) for v in self.nodes} for u in self.nodes}

    def add_link(self, u, v, cost):
        self.adj[u][v] = cost
        # initialize direct neighbor entries
        if cost < self.dv[u][v]:
            self.dv[u][v] = cost
            self.nh[u][v] = v

    def iterate(self):
        changed = False
        for u in self.nodes:
            for n, c_un in self.adj[u].items():
                for d in self.nodes:
                    cand = c_un + self.dv[n][d]
                    if cand < self.dv[u][d] - 1e-12:
                        self.dv[u][d] = cand
                        self.nh[u][d] = n if u != d else None
                        changed = True
                    elif abs(cand - self.dv[u][d]) <= 1e-12:
                        # tie-break: prefer lexicographically smaller next-hop
                        if u != d and self.nh[u][d] is not None and n < self.nh[u][d]:
                            self.nh[u][d] = n
                            changed = True
        return changed

    def run_to_convergence(self, max_rounds=1000):
        for _ in range(max_rounds):
            if not self.iterate():
                return True
        return False

    def table_for(self, u):
        rows = []
        for d in self.nodes:
            if d == u:
                continue
            cost = self.dv[u][d]
            hop = self.nh[u][d]
            rows.append((d, hop, cost))
        rows.sort(key=lambda x: x[0])
        return rows

if __name__ == "__main__":
    # --- Build Fig. 5-12 network (directed) ---
    nodes = list("ABCDEF") + ["D"]  # ensure 'D' included (already in A..F but kept explicit)
    nodes = sorted(set(nodes))

    dvnet = DistanceVector(nodes)

    # From the problem 3 (directed costs)
    dvnet.add_link("A", "B", 5)
    dvnet.add_link("A", "E", 4)

    dvnet.add_link("B", "A", 4)
    dvnet.add_link("B", "C", 1)
    dvnet.add_link("B", "F", 5)

    dvnet.add_link("C", "B", 3)
    dvnet.add_link("C", "D", 4)
    dvnet.add_link("C", "E", 3)

    dvnet.add_link("E", "A", 2)
    dvnet.add_link("E", "C", 2)
    dvnet.add_link("E", "F", 2)

    dvnet.add_link("F", "B", 1)
    dvnet.add_link("F", "D", 2)
    dvnet.add_link("F", "E", 3)

    # D's direct links (given explicitly)
    dvnet.add_link("D", "C", 3)
    dvnet.add_link("D", "F", 4)

    converged = dvnet.run_to_convergence()
    assert converged, "Did not converge (unexpected on this graph)."

    # --- Prints router D's routing table (Dest, Next-hop, Cost) ---
    rows = dvnet.table_for("D")

    print("\n+------------------------------+")
    print("| Routing Table at Router D    |")
    print("+------+----------+------------+")
    print("| Dest | Next Hop |   Cost     |")
    print("+------+----------+------------+")

    for dest, hop, cost in rows:
        cstr = "∞" if cost == INF else (int(cost) if abs(cost - int(cost)) < 1e-12 else round(cost, 3))
        hop = hop if hop is not None else "-"
        print(f"|  {dest:<3} |    {hop:<6} |    {cstr:<6} |")

    print("+------+----------+------------+")




+------------------------------+
| Routing Table at Router D    |
+------+----------+------------+
| Dest | Next Hop |   Cost     |
+------+----------+------------+
|  A   |    C      |    8      |
|  B   |    F      |    5      |
|  C   |    C      |    3      |
|  E   |    C      |    6      |
|  F   |    F      |    4      |
+------+----------+------------+
